<a href="https://colab.research.google.com/github/KumarGaurav20/Spam-Emails/blob/main/Testing_Our_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TESTING OUR MODEL-BAYES CLASSIFIER

In [5]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Notebook Imports


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

##CONSTANTS

In [4]:
TOKEN_SPAM_PROB_FILE = '/content/gdrive/MyDrive/COLAB DATA/prob-spam.txt'
TOKEN_HAM_PROB_FILE = '/content/gdrive/MyDrive/COLAB DATA/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = '/content/gdrive/MyDrive/COLAB DATA/prob-all-tokens.txt'



TEST_FEATURE_MATRIX = '/content/gdrive/MyDrive/COLAB DATA/test-features.txt'
TEST_TARGET_FILE = '/content/gdrive/MyDrive/COLAB DATA/test-target.txt'

VOCAB_SIZE = 2500

## ACESSING THE DATA

In [6]:
# Features
X_test = np.loadtxt(TEST_FEATURE_MATRIX, delimiter=' ')
# Target
y_test = np.loadtxt(TEST_TARGET_FILE, delimiter=' ')
# Token Probabilities
prob_token_spam = np.loadtxt(TOKEN_SPAM_PROB_FILE, delimiter=' ')
prob_token_ham = np.loadtxt(TOKEN_HAM_PROB_FILE, delimiter=' ')
prob_all_tokens = np.loadtxt(TOKEN_ALL_PROB_FILE, delimiter=' ')

In [7]:
X_test[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 2., 1., ..., 0., 0., 0.],
       [3., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 2., 6., ..., 0., 0., 0.]])

In [11]:
y_test[:5], y_test[-5:]   #0->NONSPAM, 1->SPAM

(array([1., 1., 1., 1., 1.]), array([0., 0., 0., 0., 0.]))

# CALCULATING THE JOINT PROBABILITY

## JOINT PROBABILIY OF SPAM


$P(Spam \, | \, X) = \frac{P(X \, | \, Spam) \, P(Spam)} {P(X)}$

PRIOR

In [16]:
# NOTE:Prior is the distribution that is imputed before the model starts to see any data.
# As we already know the prior P(Spam), we'll use that. 
PROB_SPAM 

0.3116

In [17]:
# TAKING LOG ON BOTH SIDE OF THE Eqn
joint_log_spam = X_test.dot(np.log(prob_token_spam) - np.log(prob_all_tokens)) + np.log(PROB_SPAM)


In [18]:
joint_log_spam

array([  3.21926579,  13.35310029,  19.22329985, ..., -31.24582095,
        -4.77833586, -17.03309585])

## JOINT PROBABILITY OF NONSPAM

$P(Ham \, | \, X) = \frac{P(X \, | \, Ham) \, (1-P(Spam))} {P(X)}$


In [19]:
joint_log_ham = X_test.dot(np.log(prob_token_ham) - np.log(prob_all_tokens)) + np.log(1-PROB_SPAM)

In [20]:
joint_log_ham


array([-11.05412802, -36.85845149, -38.27162954, ...,   4.83559496,
         0.28826957,   3.12480869])